<a href="https://colab.research.google.com/github/panzershracker/Recomender-systems/blob/master/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install implicit

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_csv('/content/drive/My Drive/Recomender systems/2 Урок/hw_webinar_2/retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [6]:
data.shape

(2396804, 12)

In [7]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
result_df = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_df.columns = ['user_id', 'actual']

result_df.head(3)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [9]:
weights_df = data_train.groupby('item_id')['sales_value'].sum().reset_index() #Суммы продаж по каждому товару

percent = weights_df.sales_value.quantile(0.90) #Отберем товары, которые по суммарной выручке выше 90 перцентиля

weights_df = weights_df[weights_df.sales_value >= percent]

weights_df['weights'] = np.log(weights_df.sales_value) / sum(np.log(weights_df.sales_value)) #Определяем веса каждого товара на основании суммарных покупок по нему + логарифмирование.

weights_df = weights_df.drop('sales_value', axis=1)
weights_df.head()

,item_id,weights
2892,259120,0.000110
2977,279994,0.000106
3414,397896,0.000156
3583,420647,0.000120
3880,480014,0.000132


In [10]:
data_train.shape

(2278490, 12)

In [11]:
weights_df.shape, weights_df.weights.sum()

((8687, 2), 0.9999999999999991)

In [12]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code

    probas = items_weights.weights
   
    items = np.array(items_weights.item_id)
    recs = np.random.choice(items, size=n, replace=False, p=probas)
    
    return recs.tolist()

In [13]:
%%time

# your_code

result_df['rand_weighted_recs'] = result_df['user_id'].apply(lambda x: weighted_random_recommendation(weights_df, n=5))

CPU times: user 513 ms, sys: 4.43 ms, total: 518 ms
Wall time: 526 ms


In [14]:
result_df.head(3)

,user_id,actual,rand_weighted_recs
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[825970, 7409907, 879977, 1051989, 1059823]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1133556, 996506, 968695, 831390, 930009]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1048260, 1127825, 827856, 961846, 909714]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [15]:
result = pd.read_csv('/content/drive/My Drive/Recomender systems/2 Урок/hw_webinar_2/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [16]:
def str_to_int(str_):
  
  str_ = str_[1:-1].replace('\n', '').replace(',', '').split()
  int_lst = [int(i) for i in str_]

  return int_lst

In [17]:
for row in range(result.shape[0]):
  for col in result.columns[1:]:
    result.at[row, col ]= str_to_int(result.iloc[row][col]) 

In [18]:
result['rand_weighted_recs'] = result_df['rand_weighted_recs']

In [19]:
result.head(1)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,rand_weighted_recs
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[825970, 7409907, 879977, 1051989, 1059823]"


In [20]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [21]:
for i in result.columns[2:]:
  precision = 0

  for j in result.index:
    precision += precision_at_k(result.iloc[j][i], result.iloc[j]["actual"])

  print(f'Precision_at_k for {i} is {precision/j}')

Precision_at_k for random_recommendation is 0.0005879470847623714
Precision_at_k for popular_recommendation is 0.15531602155805777
Precision_at_k for itemitem is 0.03361097501224906
Precision_at_k for cosine is 0.035276825085742466
Precision_at_k for tfidf is 0.036158745712886034
Precision_at_k for own_purchases is 0.18007512657193933
Precision_at_k for rand_weighted_recs is 0.005291523762861341


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [22]:
#  По поводу первого пункта - я не понял что нужно пересчитывать, т.к. учебном ноутбуке уже взяты 5000 товаров.
# По поводу всего 3-го задания - я пока плохо понимаю, поэтому пропущу его. Пытался - не вышло, да и работать с чужим кодом пока тяжело дается.